# Question B2 (10 marks)
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [1]:
!pip install pytorch-widedeep

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     ---------------------------------------- 44.4/44.4 kB 1.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of fuzzytm to determine which version is compatible with other requirements. This could take a while.
  Using cached torchmetrics-1.2.1-py3-none-any.whl.metadata (20 kB)
  

In [12]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd
import torch
torch.manual_seed(SEED)

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score
from torch.optim import Adam
from sklearn.metrics import mean_squared_error, r2_score

1.Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [9]:
df = pd.read_csv('hdb_price_prediction.csv')

# YOUR CODE HERE

# Step 2: Train-Test Split
# Using entries from year 2020 and before as training data, and entries from 2021 and after as test data
train_data = df[df['year'] <= 2020]
test_data = df[df['year'] >= 2021]

# Drop unused columns
train_data = train_data.drop(columns=['full_address', 'nearest_stn', 'year'])
test_data = test_data.drop(columns=['full_address', 'nearest_stn', 'year'])

2.Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [10]:
# Step 3: Configuring Features
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_cols = ['month', 'town', 'flat_model_type', 'storey_range']

# Step 4: Tab Preprocessing
preprocessor = TabPreprocessor(
    cat_embed_cols=categorical_cols,
    continuous_cols=continuous_cols
)

# Fit and transform training data, transform test data
X_train = preprocessor.fit_transform(train_data.drop(columns=['resale_price']))
X_test = preprocessor.transform(test_data.drop(columns=['resale_price']))
y_train = train_data['resale_price'].values
y_test = test_data['resale_price'].values

# Step 5: Create the TabMlp Model
model = TabMlp(
    column_idx=preprocessor.column_idx,
    cat_embed_input=preprocessor.cat_embed_input,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[200, 100],
    mlp_activation='relu'
)

# Step 6: Create WideDeep Model
wide_deep_model = WideDeep(deeptabular=model)

# Step 7: Trainer Configuration
trainer = Trainer(
    model=wide_deep_model,
    objective='rmse',
    metrics=[R2Score()],
    optimizers=Adam(wide_deep_model.parameters()),
    num_workers=0
)

# Step 8: Train the Model
trainer.fit(X_tab=X_train, target=y_train, n_epochs=100, batch_size=64)

# Step 9: Evaluate the Model on Test Data
y_pred = trainer.predict(X_tab=X_test)

C:\Users\Vaishob\anaconda3\Lib\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:360: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 79: 100%|██████████| 1366/1366 [00:23<00:00, 57.51it/s, loss=4.75e+4, metrics={'r2': 0.8995}]


predict: 100%|██████████| 1128/1128 [00:07<00:00, 152.73it/s]


3.Report the test RMSE and the test R2 value that you obtained.

In [13]:
# YOUR CODE & RESULT HERE
# Calculate RMSE and R2 on Test Set
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'TabMlp Model Test RMSE: {rmse}')
print(f'TabMlp Model Test R2: {r2}')

TabMlp Model Test RMSE: 102276.53768557492
TabMlp Model Test R2: 0.6345309827209406


C:\Users\Vaishob\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
